<a href="https://colab.research.google.com/github/MoyoM/BayessianDeepLearning/blob/master/beginner_mm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Beginner Course 1

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# install tensorflow

! pip install tensorflow==2.0.0-beta1

import tensorflow as tf

     |████████████████████████████████| 87.9MB 1.2MB/s 
     |████████████████████████████████| 3.1MB 25.5MB/s 
     |████████████████████████████████| 501kB 44.8MB/s 
  Found existing installation: tensorflow 1.14.0rc1
    Uninstalling tensorflow-1.14.0rc1:
      Successfully uninstalled tensorflow-1.14.0rc1


Load and prepare the MNIST dataset. Convert the samples from integers to floating-point numbers:

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255, x_test/255


11493376/11490434 [==============================] - 0s 0us/step


In [0]:
x_train.shape

(60000, 28, 28)

In [0]:
y_train.shape

(60000,)

Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:

In [0]:
mymodel = tf.keras.models.Sequential([
   tf.keras.layers.Flatten(input_shape=(28,28)),
   tf.keras.layers.Dense(128, activation='relu'),
   tf.keras.layers.Dropout(0.2),
   tf.keras.layers.Dense(10, activation='softmax')
])

mymodel.compile(optimizer='adam',
             loss= 'sparse_categorical_crossentropy',
             metrics =['accuracy'])


Train and Evaluate Model

In [0]:
mymodel.fit(x_train, y_train, epochs=5)

mymodel.evaluate(x_test, y_test)

W0619 10:40:05.967217 139732797126528 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 7s 123us/sample - loss: 0.2970 - accuracy: 0.9151
Epoch 2/5
60000/60000 [==============================] - 7s 111us/sample - loss: 0.1446 - accuracy: 0.9569
Epoch 3/5
60000/60000 [==============================] - 7s 110us/sample - loss: 0.1076 - accuracy: 0.9679
Epoch 4/5
60000/60000 [==============================] - 7s 112us/sample - loss: 0.0891 - accuracy: 0.9720
Epoch 5/5
10000/10000 [==============================] - 1s 53us/sample - loss: 0.0762 - accuracy: 0.9772


[0.07620418589804322, 0.9772]

*italicized text*

Expert Model

In [0]:
from tensorflow.keras.layers import Dense , Flatten, Conv2D
from tensorflow.keras import Model

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
model = MyModel()

with tf.GradientTape() as tape:
  logits = model(images)
  loss_value = loss(logits, labels)
grads = tape.gradient(loss_value, model.trainable_variables)
optimizer.apply_gradients(zip(grads, model.trainable_variables))

NameError: ignored

Class method, class inherits from keras>model. Define layers unde __init__ and forwad pass function recuersicve on data (x)

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)



In [0]:
Model = MyModel()

In [0]:
mnist2 = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist2.load_data()
x_train, x_test = x_train/255 , x_test/255

# addd nw cahnnel dimension
x_train = x_train[...,tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [0]:
# shuffle data
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Choose an optimizer and loss function for training:

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use tf.GradientTape to train the model:

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = Model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, Model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, Model.trainable_variables))
  
  train_loss(loss)
  train_accuracy(labels, predictions)
  

In [0]:
@tf.function
def test_step(images, labels):
  predictions = Model(images)
  t_loss = loss_object(labels, predictions)
  
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)
    
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
    
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Accuracy:{}'
  print (template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

Epoch 1, Loss: 0.03520289063453674, Accuracy: 98.90261840820312, Test Accuracy:0.06319214403629303
Epoch 2, Loss: 0.0313543863594532, Accuracy: 99.02062225341797, Test Accuracy:0.06451015919446945
Epoch 3, Loss: 0.028523366898298264, Accuracy: 99.10796356201172, Test Accuracy:0.06778665632009506
Epoch 4, Loss: 0.026021428406238556, Accuracy: 99.18533325195312, Test Accuracy:0.07122673839330673
Epoch 5, Loss: 0.024036232382059097, Accuracy: 99.24787902832031, Test Accuracy:0.07280152291059494
